In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe();

In [5]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name'][1:10000]))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['cozy', 'entire', 'floor', 'of', 'brownstone']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['cozy', 'entire', 'floor', 'brownstone']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [17]:
# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.182*"room" + 0.103*"entire" + 0.096*"height" + 0.078*"monthly" + '
  '0.055*"floor" + 0.053*"quiet" + 0.053*"doogler" + 0.034*"brick" + '
  '0.034*"window" + 0.024*"district"'),
 (1,
  '0.132*"light" + 0.131*"fill" + 0.131*"park" + 0.118*"classic" + '
  '0.077*"large" + 0.059*"height" + 0.044*"bright" + 0.038*"enormous" + '
  '0.025*"family" + 0.021*"east"'),
 (2,
  '0.189*"apartment" + 0.155*"sunny" + 0.112*"spacious" + 0.102*"bedroom" + '
  '0.100*"artist" + 0.100*"home" + 0.038*"beautiful" + 0.030*"height" + '
  '0.022*"meatpacke" + 0.022*"district"')]


In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.786969806724018

Coherence Score:  0.6755936831158958


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.132860  0.217725       1        1  38.469746
1     -0.204736 -0.188869       2        1  31.310192
2      0.337596 -0.028855       3        1  30.220064, topic_info=   Category         Freq         Term        Total  loglift  logprob
7   Default  1282.000000    apartment  1282.000000  30.0000  30.0000
10  Default  1571.000000         room  1571.000000  29.0000  29.0000
38  Default  1050.000000        sunny  1050.000000  28.0000  28.0000
48  Default   934.000000        light   934.000000  27.0000  27.0000
72  Default   939.000000         fill   939.000000  26.0000  26.0000
..      ...          ...          ...          ...      ...      ...
70   Topic3    12.970230  comfortable    94.409096  -0.7883  -6.2579
92   Topic3    11.046056          top    78.725113  -0.7672  -6.4185
95   Topic3    21.111952         loft   204.857101  -1.0758  -5.7708
96   Topic3    21.111952       office   204.857101  -1.0758  -5.7708
79   Topic3    12.654233     enormous   309.989441  -2.0019  -6.2826

[159 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
7         1  0.002339  apartment
7         2  0.001559  apartment
7         3  0.995647  apartment
4         1  0.090782        apt
4         2  0.907817        apt
...     ...       ...        ...
57        3  0.132050       west
78        1  0.917597     window
78        2  0.068664     window
78        3  0.012484     window
25        1  0.634414  wonderful

[242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])